In [4]:
#installing required modules
#!pip install youtube_dl
#!pip install selenium
#!pip install ffmpeg
#!pip install ffprobe

In [63]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pathlib import Path
import youtube_dl
import requests
import pandas
import os
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def DownloadVideosFromTitles(los):
    ids = []
    for index, item in enumerate(los):
        vid_id = ScrapeVidId(item)
        ids += [vid_id]
    print("Downloading songs")
    DownloadVideosFromIds(ids)

def DownloadVideosFromIds(lov):
    SAVE_PATH = str(os.path.join(Path.home(), "Downloads/songs"))
    try:
        os.mkdir(SAVE_PATH)
    except:
        print("download folder exists")
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        'outtmpl': SAVE_PATH + '/%(title)s.%(ext)s',
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(lov)

def ScrapeVidId(query):
    print ("Getting video id for: ", query)
    BASIC="http://www.youtube.com/results?search_query="
    URL = (BASIC + query)
    URL.replace(" ", "+")
    #Scraping with selenium by opening browsers and getting a list of all videos urls
    driver=webdriver.Firefox()
    driver.get(URL)
    results = [my_href.get_attribute("href") for my_href in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-video-renderer#video-title")))][0]
    return results.split('/watch?v=')[1]

'''Using a beutiful soup 
    page = requests.get(URL)
    session = HTMLSession()
    response = session.get(URL)
    response.html.arender(sleep=1)
    soup = BeautifulSoup(response.html.html, "html.parser")
    results = soup.find(id="video-title")
    print(soup)
    print(results)
    return results['href'].split('/watch?v=')[1]
'''
def __main__():
    data = pandas.read_csv('songs.csv')
    songlist = data['song names'].tolist()
    print("Found ", len(songlist), " songs!")
    #Download x amount of songs from songlist [0:x-1] , Download all songs with songlist[0:]
    DownloadVideosFromTitles(songlist[1:2])
__main__()

Found  93  songs!
Getting video id for:  I Chose Violence - iamjakehill
download folder exists


TypeError: __init__() got an unexpected keyword argument 'keepvideo'